In [17]:
def createdic(start,end):
    dic={}
    for i in range(int(start),int(end)):
        dic[i]='https://en.wikipedia.org/wiki/{}_Mosconi_Cup'.format(i)
    return(dic)

def createtables(url):
    import requests
    from lxml import html
    page = requests.get(url)
    tree = html.fromstring(page.text)
    tables = tree.xpath('//table')
    tables=[table for table in tables if(table[0].text_content().replace('\n','')=='Results')]
    return(tables)

def cdf(year,tables,df):
    import pandas as pd
    import re
    l=[]
    col=['Format','European_player','European_score','American_score','American_player']
    pattern = re.compile(r"\s?(\w+)\s(.+)\s([0-9])[–]([0-9])\s?\w+\s?(.+)\s")
    for i,table in enumerate(tables):
        for j in range(len(table)-2):
            try:
                element=tables[i][j].text_content().replace("\n", " ")
                match = re.match(pattern, element)
                if match:
                    l.append(list(match.groups()))
            except:
                pass
    if df.empty:
        df=pd.DataFrame(l,columns=col)
        df['Year']=year
        df.index.rename('Match',inplace=True)
        df.reset_index(inplace=True)
        df.set_index(['Year','Match'],inplace=True)
    else:
        new = pd.DataFrame(l, columns = col)
        new['Year']=year
        new.index.rename('Match',inplace=True)
        new.reset_index(inplace=True)
        new.set_index(['Year','Match'],inplace=True)
        frames = (df,new)
        df = pd.concat(frames)
    return(df)

def newmain():
    import pandas as pd
    dic = createdic(1998,2014)
    df = pd.DataFrame()
    for year,link in dic.items():
        tables = createtables(link)
        df = cdf(year,tables,df)
    return(df)

In [33]:
df = newmain()

In [42]:
df.head()

Format              European_player European_score  \
Year Match                                                        
1998 0      Doubles    Mika Immonen Ralf Souquet              7   
     1      Doubles  Oliver Ortmann Steve Knight              7   
     2      Doubles    Fabio Petroni Steve Davis              5   
     3      Doubles    Mika Immonen Ralf Souquet              7   
     4      Doubles    Steve Davis Fabio Petroni              3   

           American_score              American_player  
Year Match                                              
1998 0                  5    Johnny Archer Nick Varner  
     1                  6  James Rempe Earl Strickland  
     2                  7    Kim Davenport Reed Pierce  
     3                  3  James Rempe Earl Strickland  
     4                  7    Nick Varner Johnny Archer

In [44]:
df['Europe_won'] = df['European_score'] > df['American_score']
df

Format                       European_player European_score  \
Year Match                                                                 
1998 0      Doubles             Mika Immonen Ralf Souquet              7   
     1      Doubles           Oliver Ortmann Steve Knight              7   
     2      Doubles             Fabio Petroni Steve Davis              5   
     3      Doubles             Mika Immonen Ralf Souquet              7   
     4      Doubles             Steve Davis Fabio Petroni              3   
     5      Doubles           Oliver Ortmann Steve Knight              7   
     6      Singles                        Oliver Ortmann              3   
     7      Singles                          Steve Knight              3   
     8      Singles                          Mika Immonen              4   
     9      Singles                          Ralf Souquet              7   
     10     Doubles           Steve Knight Oliver Ortmann              6   
     11     Doubles             Ralf Souquet Mika Immonen              5   
     12     Doubles             Steve Davis Fabio Petroni              7   
     13     Singles                           Steve Davis              2   
     14     Singles                         Fabio Petroni              5   
     15     Singles                          Mika Immonen              4   
     16     Singles                          Ralf Souquet              7   
     17     Singles                        Oliver Ortmann              4   
     18     Singles                          Steve Knight              7   
     19     Singles                           Steve Davis              7   
     20     Singles                         Fabio Petroni              5   
     21     Singles                          Ralf Souquet              4   
1999 0      Doubles             Mika Immonen Ralf Souquet              2   
     1      Doubles              Oliver Ortmann Alex Lely              2   
     2      Doubles              Steve Davis Steve Knight              4   
     3      Doubles           Ralf Souquet Oliver Ortmann              5   
     4      Doubles             Steve Knight Mika Immonen              5   
     5      Doubles                 Steve Davis Alex Lely              5   
     6      Singles                             Alex Lely              4   
     7      Singles                          Ralf Souquet              5   
...             ...                                   ...            ...   
2012 3      Doubles    Darren Appleton Nick Ekonomopoulos              4   
     4      Singles                       Darren Appleton              5   
     5      Doubles         Darren Appleton Chris Melling              4   
     6      Singles                    Nick Ekonomopoulos              5   
     7      Doubles        Niels Feijen Nick van den Berg              5   
     8      Singles                         Chris Melling              5   
     9      Doubles          Darren Appleton Niels Feijen              4   
     10     Doubles  Nick van den Berg Nick Ekonomopoulos              5   
     11     Singles                     Nick van den Berg              4   
     12     Doubles       Nick Ekonomopoulos Niels Feijen              5   
     13     Singles                         Chris Melling              5   
     14     Singles                       Darren Appleton              5   
     15     Singles                    Nick Ekonomopoulos              3   
     16     Singles                          Niels Feijen              1   
     17     Singles                       Darren Appleton              1   
     18     Singles                         Chris Melling              5   
     19     Singles                     Nick van den Berg              5   
2013 0        Teams                           Team Europe              6   
     1      Doubles            Darren Appleton Karl Boyes              6   
     2      Singles                          Mika Immonen    

Missing 2014 data. Can get it from: http://www.matchroompool.com/mosconi-cup/mosconi-cup-xxi/#fixtures
compare stats here:
http://forums.azbilliards.com/showthread.php?t=439775&page=4

In [45]:
import re
import pandas as pd
l=[]
col = ['Format','European_player','European_score','American_score','American_player']

with open('manual_scrape_cleaned_2014.txt','r') as rf:
    for line in rf:
        match = re.match(r"(\w+)\s(.+)\s([0-9])\s?[-]\s?([0-9])\s?(.+)", line)
        if match:
            l.append(list(match.groups()))
        else:
            print('fail')
df2014 = pd.DataFrame(l,columns = col)

In [46]:
df2014['Europe_won'] = df2014['European_score'] > df2014['American_score']
df2014['Year'] = 2014
df2014.index.rename('Match',inplace=True)
df2014.reset_index(inplace=True)
df2014.set_index(['Year','Match'],inplace=True)

In [47]:
frames=[df,df2014]
df=pd.concat(frames)

2015 Data is backwards, need to fix the order (european player should be first)

In [48]:
def createdf_bw(tables):
    import re
    l=[]
    col=['Format','European_player','European_score','American_score','American_player']

    for i,table in enumerate(tables):
        for j in range(len(table)-2):
            try:
                element=tables[i][j].text_content().replace("\n", " ")
                match = re.match(r"\s?(\w+)\s(.+)\s([0-9])[–]([0-9])\s?\w+\s?(.+)\s", element)
                if match:
                    lst=[match.group(1), match.group(5), match.group(4), match.group(3), match.group(2)]
                    l.append(lst)
            except:
                pass
    df=pd.DataFrame(l,columns=col)
    return(df)

In [49]:
def main_bw():
    from lxml import html
    import requests
    import pandas as pd
    df = pd.DataFrame()
    dic = createdic(1998,2017)
    year = 2015
    link = dic[year]
    tables = createtables(link)
    newdf = createdf_bw(tables) 
    df=df.append(newdf)
    return(df)

In [50]:
df2015= main_bw()

In [51]:
df2015['Europe_won'] = df2015['European_score'] > df2015['American_score']
df2015['Year'] = 2015
df2015.index.rename('Match',inplace=True)
df2015.reset_index(inplace=True)
df2015.set_index(['Year','Match'],inplace=True)

In [52]:
frames=[df,df2015]
dfnew=pd.concat(frames)

In [54]:
pkl = 'new_inc_2015_fixed_with_years'
pd.to_pickle(dfnew,pkl)

In [56]:
def singlemain():
    import pandas as pd
    dic = createdic(2016,2017)
    df = pd.DataFrame()
    for year,link in dic.items():
        tables = createtables(link)
        df = cdf(year,tables,df)
    return(df)

In [58]:
df2016 = singlemain()
df2016['Europe_won'] = df2016['European_score'] > df2016['American_score']
df2016

Format                European_player European_score  \
Year Match                                                          
2016 0        Teams                    Team Europe              5   
     1      Doubles        Mark Gray Albin Ouschan              5   
     2      Singles                   Niels Feijen              5   
     3      Doubles    Jayson Shaw Darren Appleton              1   
     4      Singles                Darren Appleton              5   
     5      Doubles         Niels Feijen Mark Gray              5   
     6      Singles                    Jayson Shaw              3   
     7      Doubles  Albin Ouschan Darren Appleton              5   
     8      Singles                      Mark Gray              5   
     9      Doubles   Darren Appleton Niels Feijen              5   
     10     Doubles      Jayson Shaw Albin Ouschan              5   
     11     Singles                  Albin Ouschan              5   
     12     Doubles      Mark Gray Darren Appleton              4   
     13     Singles                    Jayson Shaw              5   

           American_score                    American_player Europe_won  
Year Match                                                               
2016 0                  2                           Team USA       True  
     1                  2    Shane Van Boening Rodney Morris       True  
     2                  2                      Mike Dechaine       True  
     3                  5     Skyler Woodward Justin Bergman      False  
     4                  3                  Shane Van Boening       True  
     5                  2  Shane Van Boening Skyler Woodward       True  
     6                  5                      Rodney Morris      False  
     7                  4       Mike Dechaine Justin Bergman       True  
     8                  1                     Justin Bergman       True  
     9                  2    Mike Dechaine Shane Van Boening       True  
     10                 3      Rodney Morris Skyler Woodward       True  
     11                 4                    Skyler Woodward       True  
     12                 5       Justin Bergman Rodney Morris      False  
     13                 1                      Mike Dechaine       True

In [59]:
frames=[dfnew,df2016]
dfnewer=pd.concat(frames)

In [60]:
pkl = 'new_inc_2016_fixed_again_with_years'
pd.to_pickle(dfnewer,pkl)

## Let's try to use sql instead of pickle to store the dataframe

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///all_data.sqlite')

Next steps: Rescrape the data. 

1. Need to include the year as a multiindex and concatenate the frames per year.(done)
1. Need to reconfigure the number of rows to possibly extend beyond 5. Have to dynamically learn the number of rows in the    given table and then use that number instead of hard coding it. (done)
5. Need to get data from 94-98 (done)
3. Need to collect the 'team' data each year. i.e. what players are on each team so that I can do analysis on the 'team' matches and include that in the final presentations. (done)
4. Scrape location each year and form dictionary b/w year and location (done)